In [19]:
!pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\kelln\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

#### Part a) Define Matrices and Vectors

In [21]:
# Matrix A
A = np.array([
    # Time constraints for each operation (Ingr Prep, Formulation, Packaging)
    [2.5, 2.0, 1.8, 1.2],  # Ingredient preparation times
    [12, 10, 8, 5],        # Formulation times
    [10, 8, 6, 4],         # Packaging times
    # Policy constraints
    [0, -1, 0, 1],         # Vaccine B >= Vaccine D constraint
    [0, 0, 0, 1],          # Minimum 5 batches of Vaccine D constraint
])

In [22]:
# Vector B: RHS of constraints
b =np.array([
    65, #Ingredient prep time limit
    95, #Formulation time limit
    80, #Packaging time limit
    0, #B >= D constraint
    5 #Minimum D constraint
])

In [23]:
# C: Objective function coefficients
c = np.array([20, 18, 15, 8])

#### Part B: Define and solve the problem

In [24]:
def solve_primal():
    model = gp.Model("vaccine_primal")
    
    # Create variables
    x = model.addMVar(4, vtype=GRB.CONTINUOUS, name="x")
    
    # Set objective
    model.setObjective(c @ x, GRB.MAXIMIZE)
    
    # Add constraints
    model.addConstr(A @ x <= b)
    
    # Optimize
    model.optimize()
    
    # Save model to file
    model.write("primal.lp")
    
    # Print results
    if model.status == GRB.OPTIMAL:
        print("\nPrimal Solution:")
        print(f"Optimal value: {model.objVal}")
        print("Variable values:")
        for i, var in enumerate(x):
            print(f"x{i+1}: {var.x}")
    
    return model

#### Part C: Define and solve the dual problem

In [25]:
def solve_dual():
    model = gp.Model("vaccine_dual")
    
    # Add variables (y1 through y5)
    y = model.addMVar(5, vtype=GRB.CONTINUOUS, name="y")
    
    # Set objective: Minimize b^T y
    model.setObjective(b @ y, GRB.MINIMIZE)
    
    # Add constraints
    model.addConstr(A.T @ y >= c)
    
    # Optimize
    model.optimize()
    
    # Save Model to file
    model.write("dual.lp")
    
    # Print results
    if model.status == GRB.OPTIMAL:
        print("\nDual Solution:")
        print(f"Optimal Value: {model.objVal}")
        print("Shadow Prices:")
        resources = ["Ingredient Prep", "Formulation", "Packaging", "B >= D Policy", "Min D Requirement"]
        for i, var in enumerate(y):
            print(f"y{i+1} ({resources[i]}): {var.x}")
        
        return model

#### Part D: Solve both problems

In [26]:
print("Solving Primal Problem...")
primal_model = solve_primal()

print("\nSolving Dual Problem...")
dual_model = solve_dual()

if primal_model.status == GRB.OPTIMAL and dual_model.status == GRB.OPTIMAL:
    print("\nStrong duality holds - optimal values match!")
    print(f"Primal objective value: {primal_model.objVal}")
    print(f"Dual objective value: {dual_model.objVal}")

Solving Primal Problem...
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 5 rows, 4 columns and 15 nonzeros
Model fingerprint: 0x6a714b6d
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.8500000e+02   4.861450e+01   0.000000e+00      0s
       3    1.7812500e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.781250000e+02

Primal Solution:
Optimal value: 178.125
Variable values:
x1: 0.0
x2: 0.0
x3: 11.875
x4: 0.0

Solving 